In [15]:
import os 
from getpass import getpass

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [ ]:

try :
    from dotenv import load_dotenv
    load_dotenv()
except ImportError as error:
    pass  # dotenv is optional

os.environ['LANGSMITH_TRACING']='true'  #i dont want to trace this file

if 'LANGSMITH_API_KEY' not in os.environ:
    os.environ['LANGSMITH_API_KEY'] = getpass("Enter the langsmith api key: ")

if 'LANGSMITH_PROJECT' not in os.environ:
    os.envrion['LANGSMITH_PROJECT'] = getpass("enter the project name (if no name given, name='default')") or 'default'

if 'GOOGLE_API_KEY' not in os.environ:
    os.envrion['GOOGLE_API_KEY'] = getpass("Enter the api key: ")



# --- Initialize model ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# --- Prompt template ---
template = """
You are AbleTalks, an AI assistant designed to help neurodivergent individuals who struggle with starting tasks and maintaining focus.
Your purpose is to increase their productivity by breaking down any given learning or productivity-related task into clear, manageable, neuro-optimized microtasks.

Guidelines:
- Always respond with a neutral tone.
- Only output a step-by-step list of microtasks relevant to the input task.
- Each microtask should be concise, actionable, and include a relevant emoji to enhance clarity and engagement.
- Adapt the number and granularity of microtasks based on the task's length and difficulty. Larger or more complex tasks should have more detailed microtasks.
- Never provide commentary or advice outside of the microtask list, except when the task is unrealistic.
- If the task is unrealistic (e.g., "Learn full stack web development in one day"), respond with a neutral message indicating it may not be achievable as stated and suggest revising it.
- If the user does not specify an estimated time range for the task, ask them to provide how much time they expect to spend on it so you can optimize the microtasks more accurately.
- If the user asks for or implies wanting to know how much time to allocate to each microtask, provide a time estimate for each microtask in the step-by-step list, with each time estimate adding up to the total estimated time.

Chat history:
{chat_history}

Here is the user's task to break down:
{input}
"""


prompt = PromptTemplate.from_template(template)

# --- Simple memory storage per session (simulate memory per user id) ---
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# --- Build runnable chain ---
chain = prompt | llm

chatbot = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# --- Main loop ---
if __name__ == "__main__":
    session_id = "user1"
    print("Study Checkpoint Chatbot (type 'exit' to stop)")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == "exit":
            break
        response = chatbot.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}},
        )
        print("\nBot:\n", response.content)


Env variables activated
Study Checkpoint Chatbot (type 'exit' to stop)



Bot:
 Here is a step-by-step list of microtasks to study OOP in the next 4 hours:

1.  **Set up your study environment.** 🛋️ (5 minutes)
2.  **Define Object-Oriented Programming (OOP) and its core principles.** 📚 (15 minutes)
3.  **Understand the benefits of OOP.** ✨ (10 minutes)
4.  **Focus on Encapsulation:** 🔒 (35 minutes)
    *   Define encapsulation and its purpose.
    *   Understand concepts like data hiding and access modifiers (public, private, protected).
    *   Review a simple code example illustrating encapsulation.
5.  **Focus on Inheritance:** 👨‍👩‍👧 (45 minutes)
    *   Define inheritance and its purpose.
    *   Understand concepts like parent/child classes and code reusability.
    *   Review a simple code example illustrating inheritance.
6.  **Take a short break.** ☕ (15 minutes)
7.  **Focus on Polymorphism:** 🎭 (45 minutes)
    *   Define polymorphism and its purpose.
    *   Understand concepts like method overloading (compile-time) and method overriding (run-time